In [1]:
import numpy as np
import threading
import time

class SimAxis:
    """Simulate a stepper motor."""

    readback = 0.
    setpoint = 0.
    tolerance = 0.001
    noise_amplitude = 0.0002
    done = True
    low_limit = -1
    high_limit = 121
    units = "mm"
    _poll_interval_s = 0.005
    _max_move = 0.05
    _stopping = False
    _controller_thread = None

    def __init__(self, prefix, name=None):
        self.prefix = prefix
        if name is None:
            raise ValueError("Must specify keyword argument 'name' with a string value.")
        self.name = name

    def __repr__(self, *args, **kwargs):
        nm = self.__class__.__name__
        arglist = [f'"{self.prefix}"',]
        for attr in "name readback setpoint units".split():
            v = getattr(self,attr)
            if isinstance(v, str):
                arglist.append(f'{attr}="{v}"')
            else:
                arglist.append(f"{attr}={v}")
        return f"{nm}({', '.join(arglist)})"

    def __str__(self, *args, **kwargs):
        return f"{self.name} = {self.position:.3f} {self.units}"

    def check_limits(self, value):
        if value < self.low_limit:
            raise ValueError(f"{value} is less than low limit: {self.limits}")
        elif value > self.high_limit:
            raise ValueError(f"{value} is greater than high limit: {self.limits}")

    def _controller(self, *value, **kwargs):
        # print(f"{self.moving = }")
        while self.moving:
            if self._stopping:
                print(f"stopping ...  {self.readback = }")
                self._stopping = False
                self.setpoint = self.readback
                break
            if not self.done:
                self._step()
            time.sleep(self._poll_interval_s)
            # print(f"{self.moving = }")
        self.done = True
        self._controller_thread = None

    @property
    def limits(self):
        return (self.low_limit, self.high_limit)

    def move(self, value):
        self.check_limits(value)
        # print(f"move({value})")
        self.setpoint = float(value)
        self.done = False
        runner = threading.Thread(target=self._controller)
        runner.start()
        self._controller_thread = runner

    @property
    def moving(self):
        diff = float(self.readback) - float(self.setpoint)
        return abs(diff) > self.tolerance

    @property
    def position(self):
        return self.readback

    def stop(self):
        if not self.done:
            print("stop received")
            self._stopping = True

    def _step(self):
        """simulate a motion"""
        diff = self.setpoint - self.readback
        sign = {True: 1, False: -1}[diff >= 0]
        step = sign*min(abs(diff)/2, self._max_move)
        noise = self.noise_amplitude*np.random.normal()
        target = step+noise
        # print(f"{diff = }")
        # print(f"{step = }")
        # print(f"{noise = }")
        # print(f"{target = }")

        self.readback += max(min(target, self.high_limit), self.low_limit)
        # print(f"{self.readback = }")


def move(axis, value, wait=True):
    axis.move(value)
    if not wait:
        return 0
    t0 = time.time()
    while not axis.done:
        time.sleep(0.1)
    return time.time()-t0

In [2]:
x = SimAxis("", name="x")
print(f"{x!r}")

SimAxis("", name="x", readback=0.0, setpoint=0.0, units="mm")


In [3]:
print(move(x, 1), "s")
print(repr(x))

print(move(x, 10), "s")
print(repr(x))

0.20122718811035156 s
SimAxis("", name="x", readback=0.9990462720750193, setpoint=1.0, units="mm")
1.1054065227508545 s
SimAxis("", name="x", readback=9.999113035336817, setpoint=10.0, units="mm")


In [4]:
# test the limits
for v in (-12_345, 543_212):
    try:
        x.move(v)
    except ValueError as exc:
        print(exc)

-12345 is less than low limit: (-1, 121)
543212 is greater than high limit: (-1, 121)


In [5]:
# relative move, don't wait
move(x, x.position+10, wait=False)

# let it run a short time
time.sleep(1.5)

# stop the move, report how long it takes to stop
t0 = time.time()
x.stop()
while not x.done:
    time.sleep(0.01)
print(f"time to stop: {time.time()-t0:.3f}s  {x!r}")

time to stop: 0.000s  SimAxis("", name="x", readback=19.998251892442408, setpoint=19.99911303533682, units="mm")


In [6]:
# a series of moves
for v in range(10, 1, -1):
    print(f"{move(x, v):.3f}s  {x!r}")

1.306s  SimAxis("", name="x", readback=10.000133289217729, setpoint=10.0, units="mm")
0.200s  SimAxis("", name="x", readback=9.000916413607326, setpoint=9.0, units="mm")
0.201s  SimAxis("", name="x", readback=8.000538170122626, setpoint=8.0, units="mm")
0.202s  SimAxis("", name="x", readback=7.000647568391116, setpoint=7.0, units="mm")
0.202s  SimAxis("", name="x", readback=6.0006293735319804, setpoint=6.0, units="mm")
0.201s  SimAxis("", name="x", readback=5.000686029981178, setpoint=5.0, units="mm")
0.201s  SimAxis("", name="x", readback=4.00063048424907, setpoint=4.0, units="mm")
0.201s  SimAxis("", name="x", readback=3.00056808303173, setpoint=3.0, units="mm")
0.201s  SimAxis("", name="x", readback=2.00092410331486, setpoint=2.0, units="mm")


In [7]:
# measure the movement speed for short and long moves
print(repr(x))
for hi in (-0.1, 1, 2, 5, 10, 20, 30):
    for v in (hi, 0):
        v0 = x.position
        dt = move(x, v)
        dx = x.position - v0
        s = dx/dt
        print(f"{dt:.3f}s  {dx = :.2f}{x.units}  {s = :.3f}{x.units}/s")

SimAxis("", name="x", readback=2.00092410331486, setpoint=2.0, units="mm")
0.301s  dx = -2.10mm  s = -6.972mm/s
0.100s  dx = 0.10mm  s = 0.980mm/s
0.201s  dx = 1.00mm  s = 4.975mm/s
0.201s  dx = -1.00mm  s = -4.976mm/s
0.301s  dx = 2.00mm  s = 6.631mm/s
0.302s  dx = -2.00mm  s = -6.625mm/s
0.705s  dx = 5.00mm  s = 7.095mm/s
0.702s  dx = -5.00mm  s = -7.120mm/s
1.306s  dx = 10.00mm  s = 7.655mm/s
1.307s  dx = -10.00mm  s = -7.652mm/s
2.413s  dx = 20.00mm  s = 8.288mm/s
2.412s  dx = -20.00mm  s = -8.293mm/s
3.620s  dx = 30.00mm  s = 8.287mm/s
3.620s  dx = -30.00mm  s = -8.287mm/s
